In [1]:
from pathlib import Path
import czifile
import napari
import numpy as np
from scipy.ndimage import binary_erosion
import pyclesperanto_prototype as cle


In [2]:
directory_path = Path("./raw_data/NeuN_Calbinding_KI67")

In [3]:
# Create an empty list to store all image filepaths within the dataset directory
images = []

# Iterate through the lsm files in the directory
for file_path in directory_path.glob("*.czi"):
    images.append(str(file_path))
    
images

['raw_data\\NeuN_Calbinding_KI67\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\NeuN_Calbinding_KI67\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

In [4]:
image = images[0]

# Read path storing raw image and extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file
img = czifile.imread(image)

In [5]:
# Remove singleton dimensions
img = img.squeeze()
img.shape

(4, 14, 3803, 2891)

In [6]:
# Define the nuclei and markers of interest channels ('Remember in Python one starts counting from zero')
nuclei_channel = 3

nuclei_stack = img[nuclei_channel, :, :, :]

# Perform maximum intensity projections
nuclei_mip = np.max(nuclei_stack, axis=0)

nuclei_mip.shape

(3803, 2891)

In [7]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)
viewer.add_image(nuclei_stack)
viewer.add_image(nuclei_mip)

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


<Image layer 'nuclei_mip' at 0x20a81b23d00>

In [8]:
from skimage import measure, exposure, filters
from cellpose import models

model = models.Cellpose(gpu=True, model_type="nuclei")

In [9]:
gaussian_sigma = 0
cellpose_nuclei_diameter = None

In [10]:
# Create a copy of nuclei_mip
input_img = nuclei_mip.copy()

# Might need to perform a Gaussian-blur before
post_gaussian_img = filters.gaussian(
    input_img, sigma=gaussian_sigma
)

# Apply Contrast Stretching to improve Cellpose detection of overly bright nuclei
p2, p98 = np.percentile(post_gaussian_img, (2, 98))
img_rescale = exposure.rescale_intensity(
    post_gaussian_img, in_range=(p2, p98)
)

# Predict nuclei nuclei_masks using cellpose
nuclei_masks, flows, styles, diams = model.eval(
    img_rescale,
    diameter=cellpose_nuclei_diameter,
    channels=[0, 0],
    net_avg=False,
)

In [11]:
viewer.add_labels(nuclei_masks)

<Labels layer 'nuclei_masks' at 0x20a83a23d00>

In [12]:
# Define your marker of interest channel
marker_channel = 1
marker_channel_threshold = 100
erosion_factor = 3

marker_stack = img[marker_channel, :, :, :]

# Perform maximum intensity projections
marker_mip = np.max(marker_stack, axis=0)

viewer.add_image(marker_mip)


<Image layer 'marker_mip' at 0x20aabad88e0>

In [13]:
# Erode dilated_nuclei_masks to obtain separate objects upon binarization
eroded_nuclei_masks = cle.erode_labels(nuclei_masks, radius=1)

# Set a threshold value for the pixels in microglia channel
threshold = marker_channel_threshold  # Based on the microglia marker intensity across images

# Create a new array with the same shape as nuclei_masks, initialized with False
result_array = np.full_like(nuclei_masks, False, dtype=bool)

# Find indices where values in values_array are above the threshold
above_threshold_indices = marker_mip > threshold

# Update the corresponding positions in the result_array based on the mask_array
nuclei_masks_bool = nuclei_masks.astype(
    bool
)  # Make a boolean copy of nuclei_masks to be able to use logical operators
result_array[nuclei_masks_bool & above_threshold_indices] = True

# Convert the boolean array to a binary array
binary_result_array = result_array.astype(int)

# Now, result_array contains True where both conditions are satisfied, and False otherwise
# viewer.add_labels(binary_result_array, name=f"{filename}_nuclei+glia_coloc")

# Erode binary_result_array to get rid of small nuclei pixels colocalizing with glia branches
# Set the structuring element for erosion
structuring_element = np.ones(
    (
        erosion_factor,
        erosion_factor,
    ),
    dtype=bool,
)  # You can adjust the size and shape

# Perform binary erosion
eroded_array = binary_erosion(
    binary_result_array, structure=structuring_element
)

# Now I want to recover just the nuclei_masks that are sitting on top of binary_results_array
labels, num_labels = measure.label(nuclei_masks, return_num=True)

# Create an array of indices corresponding to the True values in binary_result_array
true_indices = np.where(eroded_array)

# Initialize an array to store labels for each processed region
processed_region_labels = np.zeros_like(nuclei_masks, dtype=int)

# Label index for processed regions
label_index = 1

# Iterate over each connected component
for label_ in range(1, num_labels + 1):
    # Extract the region corresponding to the current label
    region = labels == label_

    # Check if any True value in binary_result_array is present in the region
    if np.any(region[true_indices]):
        # Assign a unique label to the processed region
        processed_region_labels[region] = label_index
        label_index += 1

viewer.add_labels(processed_region_labels, name=f"{filename}_marker+_nuclei")


<Labels layer 'HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1_marker+_nuclei' at 0x20aabcb54c0>